In [1]:
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from matplotlib import pyplot as plt
from datetime import datetime, timedelta
from tensorflow.keras.utils import plot_model
from keras.models import Sequential
from keras.layers import LSTM, Dense
from functools import reduce
from tqdm import tqdm

2023-12-13 16:24:37.869676: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-13 16:24:37.969753: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-13 16:24:37.996269: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-13 16:24:38.474911: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

#### 데이터 준비

In [2]:
CNBC = pd.read_csv("/home/user/exercise_j/GPT Project/AutoDataModify/test/CNBCTESTFinal_classified.csv")
SA = pd.read_csv("/home/user/exercise_j/GPT Project/AutoDataModify/test/SATESTFinal_classified.csv")

In [3]:
CNBC.dropna(subset=['date', 'ticker', 'sentiment_score'], inplace=True)
SA.dropna(subset=['date', 'ticker', 'sentiment_score'], inplace=True)

In [4]:
CNBC['date'] = pd.to_datetime(CNBC['date'])
SA['date'] = pd.to_datetime(SA['date'])

CNBC.sort_values(by='date', ascending=True, inplace=True)
SA.sort_values(by='date', ascending=True, inplace=True)

In [5]:
CNBC.reset_index(drop=True, inplace=True)
SA.reset_index(drop=True, inplace=True)

In [6]:
del CNBC['ticker']
del SA['ticker']

In [7]:
# 2106년과 2006년 데이터 필터링
SA['date'] = pd.to_datetime(SA['date'])
SA = SA[(SA['date'].dt.year != 2106) & (SA['date'].dt.year != 2024) &  ~((SA['date'].dt.year == 2023) & (SA['date'].dt.month == 12)) & (SA['date'].dt.year != 2008) & (SA['date'].dt.year != 2006) & (SA['date'].dt.year != 2007) & (SA['date'].dt.year != 2008)]

#### Seeking Alpha(SA)와 CNBC 데이터셋을 병합하여 입력 데이터셋을 구성합니다.

In [8]:
# 두 데이터프레임을 합치기
combined_df = pd.concat([CNBC, SA], ignore_index=True)

print(combined_df.head())

                                             Content       date  \
0  Ash Bennington   11/30/2010 3:13:56 AMFA PLAYB... 2010-11-30   
1  John Carney   6/21/2011 5:51:34 AMInvestors ur... 2011-06-21   
2  Matt Clinch   7/8/2013 3:10:37 PMCritics Unfri... 2013-07-08   
3  Seema Mody   7/9/2013 3:29:01 AMBitcoin ETF pl... 2013-07-09   
4  Arash Massoudi and Tracy Alloway   7/16/2013 8... 2013-07-16   

   sentiment_score content  
0              0.5     NaN  
1              0.5     NaN  
2              0.5     NaN  
3              0.2     NaN  
4              0.7     NaN  


In [9]:
combined_df_sum = combined_df.groupby('date')['sentiment_score'].sum().reset_index()
print(combined_df_sum)

           date  sentiment_score
0    2010-08-08              0.2
1    2010-11-30              0.5
2    2011-01-07              0.6
3    2011-06-21              0.5
4    2012-12-17              0.7
...         ...              ...
2018 2023-11-16              0.5
2019 2023-11-17              0.2
2020 2023-11-20              2.0
2021 2023-11-21              0.6
2022 2023-11-23              0.7

[2023 rows x 2 columns]


##### 2023년 11월 23일에 'crypto' 카테고리의 호재인 뉴스가 많았음을 추측해볼 수 있습니다.

In [10]:
combined_df_sum.max()

date               2023-11-23 00:00:00
sentiment_score                   26.5
dtype: object

#### BTC 거래 종가, 거래 규모 데이터 (2010.11.30 ~ 2023.11.30)

In [11]:
BTC = pd.read_csv("/home/user/exercise_j/GPT Project/LSTM/BTC_historical_dataset.csv")

In [12]:
del BTC['변동 %']
del BTC['고가']
del BTC['시가']
del BTC['저가']
BTC.tail()

,날짜,종가,거래량
4744,2010- 12- 04,0.2,28.56K
4745,2010- 12- 03,0.3,2.57K
4746,2010- 12- 02,0.3,20.31K
4747,2010- 12- 01,0.2,8.16K
4748,2010- 11- 30,0.2,21.48K


#### (${p_t}, {v_t}, {s_t}$) 만들기

In [13]:
combined_df_sum['date'] = pd.to_datetime(combined_df_sum['date'])
BTC['date'] = pd.to_datetime(BTC['날짜'])
del BTC['날짜']

In [14]:
df_merged = pd.merge(BTC, combined_df_sum, on='date', how='outer')

# 결과 확인
print(df_merged)

            종가     거래량       date  sentiment_score
0     37,712.9  33.53K 2023-11-30              NaN
1     37,855.5  49.34K 2023-11-29              NaN
2     37,823.3  57.50K 2023-11-28              NaN
3     37,248.6  45.24K 2023-11-27              NaN
4     37,451.8  29.20K 2023-11-26              NaN
...        ...     ...        ...              ...
4745       0.3   2.57K 2010-12-03              NaN
4746       0.3  20.31K 2010-12-02              NaN
4747       0.2   8.16K 2010-12-01              NaN
4748       0.2  21.48K 2010-11-30              0.5
4749       NaN     NaN 2010-08-08              0.2

[4750 rows x 4 columns]


#### 'sentiment_score'의 NaN 값은 뉴스가 없는 상태로 0.5(중립)으로 치환

In [15]:
df_merged['sentiment_score'].fillna(0.5, inplace=True)

# 결과 확인
print(df_merged.head())

         종가     거래량       date  sentiment_score
0  37,712.9  33.53K 2023-11-30              0.5
1  37,855.5  49.34K 2023-11-29              0.5
2  37,823.3  57.50K 2023-11-28              0.5
3  37,248.6  45.24K 2023-11-27              0.5
4  37,451.8  29.20K 2023-11-26              0.5


#### 결측치 제거 및 확인

In [16]:
df_merged.dropna(inplace=True)
df_merged.reset_index(drop=True, inplace=True)
df_merged.isna().sum()

종가                 0
거래량                0
date               0
sentiment_score    0
dtype: int64

#### BTC 거래량 데이터 변환

In [17]:
def convert_volume(volume):
    if isinstance(volume, str):  # volume이 문자열인지 확인
        if 'K' in volume:
            return float(volume.replace('K', '')) * 1000
        elif 'M' in volume:
            return float(volume.replace('M', '')) * 1000000
        elif 'B' in volume:
            return float(volume.replace('B', '')) * 1000000000
        else:
            return float(volume)
    else:
        return volume

df_merged['거래량'] = df_merged['거래량'].apply(convert_volume)


In [18]:
# NumPy 배열로 변환
prices = df_merged['종가'].values.reshape(-1, 1)
volumes = df_merged['거래량'].values.reshape(-1, 1)
sentiments = df_merged['sentiment_score'].values.reshape(-1, 1)

# NumPy 배열을 (p_t, v_t, s_t) 꼴로 결합
data = np.hstack((prices, volumes, sentiments))

# 결과 확인
print(data)

[['37,712.9' 33530.0 0.5]
 ['37,855.5' 49340.0 0.5]
 ['37,823.3' 57500.0 0.5]
 ...
 ['0.3' 20310.0 0.5]
 ['0.2' 8160.0 0.5]
 ['0.2' 21480.0 0.5]]


#### GPU 설정 확인 및 조정

In [19]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  2


In [20]:
import tensorflow as tf

physical_devices = tf.config.list_physical_devices('GPU')

# 모든 GPU에 대해 메모리 성장 설정 적용
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)


#### LSTM 모델 설정

In [21]:
### LSTM 모델 불러오기
input_data = np.reshape(data, (1, 1, -1))
input_size = len(prices[0]) + len(volumes[0]) + len(sentiments[0])
lstm_units = 50 # 초기 셀의 개수 설정

def initialize_ecm_lstm(input_size, lstm_units):
    model = Sequential()
    model.add(LSTM(lstm_units, input_shape=(None, input_size)))
    model.add(Dense(1))  # 예측할 값이 1차원 (예: 수익률)
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

model = initialize_ecm_lstm(input_size, lstm_units)

2023-12-13 16:24:39.770664: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-13 16:24:40.599994: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22283 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:1a:00.0, compute capability: 8.6
2023-12-13 16:24:40.600617: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22267 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:68:00.0, compute capability: 8.6


### **ECM-LSTM 알고리즘**

In [27]:
centroids = []
centroids.append(data[0])
radiuses = []
radiuses.append(0)

def convert_to_float(arr):
    return np.array([float(str(val).replace(',', '')) if isinstance(val, str) else val for val in arr])

# 모든 데이터를 문자열 => 숫자로 변환
data = np.array([convert_to_float(row) for row in data])
centroids = [convert_to_float(c) for c in centroids]

### ECM-LSTM 알고리즘

def initial_step(data, centroids, radiuses, model):
    predicted_returns = []

    for t in tqdm(range(len(data)), desc="Processing"):
        new_data = np.concatenate([prices[t], volumes[t], sentiments[t]])
        new_data = convert_to_float(new_data) 
        closest_idx = None
        min_distance = float('inf')
        s_min_init = float('inf')

        # 가장 가까운 클러스터 찾기
        for idx, c in enumerate(centroids):
            distance = np.linalg.norm(new_data - c)
            if distance < min_distance:
                min_distance = distance
                closest_idx = idx

        # 클러스터 업데이트
        if closest_idx is not None and closest_idx < len(radiuses) and min_distance <= radiuses[closest_idx]:
            centroids[closest_idx] = (centroids[closest_idx] + new_data) / 2
            radiuses[closest_idx] = max(radiuses[closest_idx], min_distance)
        else:
            # 가장 가까운 클러스터 찾기
            for idx, (c, r) in enumerate(zip(centroids, radiuses)):
                s_min = np.linalg.norm(new_data - c) + r
                if s_min < s_min_init : 
                    s_min_init = s_min
                    closest_idx = idx

            if closest_idx is not None and s_min > 2 * radiuses[closest_idx]:
                centroids.append(new_data)
                
            else :
                centroids[closest_idx] = (centroids[closest_idx] + new_data) / 2
                radiuses[closest_idx] = max(radiuses[closest_idx], min_distance)
        # LSTM 모델을 사용한 예측
        predicted_return = model.predict(np.reshape(new_data, (1, 1, -1)))
        predicted_returns.append(predicted_return.tolist())
        print(predicted_return)
    print(predicted_returns)
    return centroids, radiuses, predicted_returns

In [29]:
centroids, radiuses, predicted_returns = initial_step(data, centroids, radiuses, model)

Processing:   0%|          | 0/4743 [00:00<?, ?it/s]

1/1 [==============================] - 0s 21ms/step
[[-0.04723405]]
1/1 [==============================] - 0s 18ms/step


Processing:   0%|          | 2/4743 [00:00<05:10, 15.26it/s]

[[0.4951876]]
1/1 [==============================] - 0s 19ms/step
[[0.40145895]]
1/1 [==============================] - 0s 17ms/step


Processing:   0%|          | 4/4743 [00:00<04:59, 15.82it/s]

...


1/1 [==============================] - 0s 17ms/step
[[0.02383865]]
1/1 [==============================] - 0s 17ms/step


Processing: 100%|█████████▉| 4741/4743 [04:49<00:00, 14.70it/s]

[[0.02500401]]
1/1 [==============================] - 0s 16ms/step
[[0.0250038]]
1/1 [==============================] - 0s 17ms/step


Processing: 100%|██████████| 4743/4743 [04:49<00:00, 16.38it/s]

[[0.02500401]]
[[[-0.047234050929546356]], [[0.4951876103878021]], [[0.4014589488506317]], [[0.4951876103878021]], [[0.20109179615974426]], [[0.42509692907333374]], [[0.35693466663360596]], [[0.04338831827044487]], [[0.35693466663360596]], [[0.35693466663360596]], [[0.8286309838294983]], [[0.1577940285205841]], [[0.25837525725364685]], [[0.8286309838294983]], [[0.35693466663360596]], [[0.35693350434303284]], [[0.35693466663360596]], [[0.4951876103878021]], [[0.25837525725364685]], [[0.29172971844673157]], [[0.4014589488506317]], [[0.14285540580749512]], [[0.4951876103878021]], [[0.4014589488506317]], [[0.289705365896225]], [[0.039055418223142624]], [[0.1577940285205841]], [[0.4014589488506317]], [[0.35693466663360596]], [[0.26651695370674133]], [[0.4951876103878021]], [[0.4951876103878021]], [[0.29172971844673157]], [[0.302899569272995]], [[0.8286309838294983]], [[0.356952041387558]], [[0.20445802807807922]], [[0.2302207350730896]], [[0.2302207350730896]], [[0.44943320751190186]], [[0.

#### 각 t 시점(하루 단위) 예상 수익률 $Q_t$ => 리스트 형태인 'predicted_returns'에 저장

In [34]:
print(predicted_returns)

[[[-0.047234050929546356]], [[0.4951876103878021]], [[0.4014589488506317]], [[0.4951876103878021]], [[0.20109179615974426]], [[0.42509692907333374]], [[0.35693466663360596]], [[0.04338831827044487]], [[0.35693466663360596]], [[0.35693466663360596]], [[0.8286309838294983]], [[0.1577940285205841]], [[0.25837525725364685]], [[0.8286309838294983]], [[0.35693466663360596]], [[0.35693350434303284]], [[0.35693466663360596]], [[0.4951876103878021]], [[0.25837525725364685]], [[0.29172971844673157]], [[0.4014589488506317]], [[0.14285540580749512]], [[0.4951876103878021]], [[0.4014589488506317]], [[0.289705365896225]], [[0.039055418223142624]], [[0.1577940285205841]], [[0.4014589488506317]], [[0.35693466663360596]], [[0.26651695370674133]], [[0.4951876103878021]], [[0.4951876103878021]], [[0.29172971844673157]], [[0.302899569272995]], [[0.8286309838294983]], [[0.356952041387558]], [[0.20445802807807922]], [[0.2302207350730896]], [[0.2302207350730896]], [[0.44943320751190186]], [[0.460397809743881